In [4]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

In [8]:
def preprocess_datasets(base_path, languages, img_size=(64, 64)):
    X, y, language_ids = [], [], []
    
    for lang_id, language in enumerate(languages):
        lang_path = os.path.join(base_path, language)
        
        for alphabet_class in os.listdir(lang_path):
            class_path = os.path.join(lang_path, alphabet_class)
            
            if not os.path.isdir(class_path):
                continue
                
            for img_file in os.listdir(class_path):
                img_path = os.path.join(class_path, img_file)
                
                try:
                    # Read and preprocess image
                    img = cv2.imread(img_path)
                    if img is None:
                        continue
                        
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, img_size)
                    
                    # Normalize pixel values
                    img = img / 255.0
                    
                    X.append(img)
                    y.append(alphabet_class)
                    language_ids.append(lang_id)
                except Exception as e:
                    print(f"Error processing {img_path}: {e}")
    
    return np.array(X), np.array(y), np.array(language_ids)



In [10]:
# Languages in your dataset
languages = [
    "America",
    "Filipino",
    "India",
    "Indonesia",
    "Malaysia"
]

X, y, language_ids = preprocess_datasets(r"C:\Users\User\minor_project\dataset", languages)

# Split dataset
X_train, X_test, y_train, y_test, lang_train, lang_test = train_test_split(
    X, y, language_ids, test_size=0.2, random_state=42, stratify=y
)

In [11]:
# Create data generator for augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [12]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Concatenate, Dropout,Embedding,Reshape

set_global_policy('mixed_float16')

def build_sign_language_model(num_classes, num_languages, img_size=(64, 64, 3)):
    
    # Input layer
    input_img = Input(shape=img_size, name="image_input")
    input_language = Input(shape=(1,), name="language_input")
    
    # Base CNN model (MobileNetV2)
    base_model = MobileNetV2(
        input_shape=img_size,
        include_top=False,
        weights='imagenet'
    )
    
    # Freeze early layers
    for layer in base_model.layers[:100]:
        layer.trainable = False
    
    # Image feature extraction
    x = base_model(input_img)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    
    # Concatenate language ID as a one-hot encoded vector
    language_embedding = Embedding(input_dim=num_languages, output_dim=num_languages)(input_language)
    language_one_hot = Reshape((num_languages,))(language_embedding)
    
    # Combine image features with language context
    combined = Concatenate()([x, language_one_hot])
    
    # Classification layers
    x = Dense(512, activation='relu')(combined)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    output = Dense(num_classes, activation='softmax', dtype="float16")(x)
    
    # Create and compile model
    model = Model(inputs=[input_img, input_language], outputs=output)
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model


In [13]:
# Convert labels to numerical indices
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Build and train the model
alphabet_classes = len(label_encoder.classes_)
model = build_sign_language_model(alphabet_classes, len(languages))

# Train the model
history = model.fit(
    [X_train, lang_train],
    y_train_encoded,
    validation_data=([X_test, lang_test], y_test_encoded),
    epochs=30,
    batch_size=8,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(factor=0.2, patience=3)
    ]
)

# Save the model for later use
model.save("sign_language_model2.h5")

C:\Users\User\AppData\Local\Temp\ipykernel_20656\3969826140.py:15: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/30
4130/5989 ━━━━━━━━━━━━━━━━━━━━ 36:09 1s/step - accuracy: 0.0401 - loss: 6.7225   

KeyboardInterrupt: 